# 01 Data Summary

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
data = pd.read_excel("../data/raw_data.xlsx")

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
treatment_protocol_col = '治疗方案（药物和剂量）'
df = data.join(pd.DataFrame(mlb.fit_transform(data.pop(treatment_protocol_col)),
                          columns=mlb.classes_,
                          index=data.index))
df.to_csv("../result/00pre-processing/01base_data.csv", index=False, encoding="utf-8-sig")

# 02 Missing Data Processing

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
data = pd.read_csv("../result/00pre-processing/01base_data.csv")
missing_ratio = data.isna().mean()
selected_features = missing_ratio[missing_ratio < 0.4].index
final_data = data[selected_features]
final_data.drop(columns=['name'],inplace=True)

## Compare Imputation Methods

In [ ]:
df = final_data.copy(deep=True)

import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.impute import IterativeImputer
from missingpy import MissForest
from src.common import setup_seed
import seaborn as sns
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

df_no_miss = df.dropna()
df_no_miss_no_label = df_no_miss.drop(['half-year label','one-year label'],axis=1)
nolabel_df = df.drop(['half-year label','one-year label'],axis=1)
missing_ratio = nolabel_df.isna().mean()
row_num, index_num = df_no_miss_no_label.shape

miss_num = round(row_num * missing_ratio, 0)

In [ ]:
# log_mse definition
def log_mse(original, imputed, epsilon=1e-8):
    log_original = np.log(original + epsilon)
    log_imputed = np.log(imputed + epsilon)
    return ((log_original - log_imputed) ** 2).mean(axis=0)

# Imputation ways
strategies = {
    "Mean": SimpleImputer(strategy="mean"),
    "Zero": SimpleImputer(strategy="constant", fill_value=0),
    "Median": SimpleImputer(strategy="median"),
    "Mode": SimpleImputer(strategy="most_frequent"),
    "KNN": KNNImputer(n_neighbors=5),
    "Iter": IterativeImputer(random_state=42),
    "RF": MissForest(criterion='squared_error',max_features='sqrt',max_iter=100,verbose=0),
}

setup_seed(42)
n_experiments = 5
results_modified_msre = {strategy: [] for strategy in strategies}
results_log_mse = {strategy: [] for strategy in strategies}

for _ in range(n_experiments):
    init_df = df_no_miss_no_label.copy(deep=True)
    for feature, missing_count in miss_num.items():
        if int(missing_count)==0:
            continue
        else:
            missing_indices = random.sample(list(df_no_miss_no_label.index), int(missing_count))
            init_df.loc[missing_indices, feature] = np.nan

    for name, imputer in strategies.items():
        df_imputed = pd.DataFrame(imputer.fit_transform(init_df), columns=init_df.columns, index=init_df.index)
        log_mse_score = log_mse(df_no_miss_no_label, df_imputed).mean()
        results_log_mse[name].append(log_mse_score)

log_mse_means = {name: np.mean(scores) for name, scores in results_log_mse.items()}
log_mse_stds = {name: np.std(scores) for name, scores in results_log_mse.items()}

In [ ]:
plt.figure(figsize=(12,7), facecolor='none')
plt.rcParams['font.family'] = 'Arial'

methods = list(log_mse_means.keys())
means = np.array(list(log_mse_means.values()))
stds = np.array(list(log_mse_stds.values()))

sorted_indices = np.argsort(means)[::-1]
methods = np.array(methods)[sorted_indices]
means = means[sorted_indices]
stds = stds[sorted_indices]
ax = sns.barplot(x=means, y=methods, palette="viridis", orient='h',linewidth=1,edgecolor='black',width=0.6)

# Add error bar
for i, (mean, std) in enumerate(zip(means, stds)):
    ax.errorbar(mean, i, xerr=std, fmt='none', c='black', capsize=5, elinewidth=1.5)
    if i == len(means) - 1:
        ax.axvline(x=mean, color='gray', linestyle='--', label='min LogMSE',linewidth=2.5)
        ax.legend(loc="lower right",prop={'weight': 'bold', 'size': 12})

# Add text annotation
for i, (mean, std) in enumerate(zip(means, stds)):
    ax.text(mean + std+0.05, i, f'{mean:.2f}±{std:.2f}', va='center', c='black')

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight('bold')
# plt.title("Log MSE Evaluation", fontsize=16, fontweight='bold', c='k')
plt.xlabel('Log MSE', fontsize=14, fontweight='bold', c='k')
plt.ylabel("Imputation Method", fontsize=14, fontweight='bold', c='k')

plt.tick_params(axis='x', colors='k', labelsize=12)
plt.tick_params(axis='y', colors='k', labelsize=12)
ax.set_xlim(0, 14)
ax = plt.gca()  # Obtain the current axis
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_linewidth(2) # bold the x-axis
ax.spines['left'].set_linewidth(2)

ax.xaxis.set_tick_params(width=2) # bold the scale
ax.yaxis.set_tick_params(width=2)
ax.set_facecolor('none')
plt.tight_layout()

plt.savefig("../result/00pre-processing/02impute_way_compare_origin_miss_ratio.pdf",dpi=300,bbox_inches="tight",format='pdf')
plt.savefig("../result/00pre-processing/02impute_way_compare_origin_miss_ratio.tif",dpi=300,bbox_inches="tight",format='tif')
# plt.show()

## Impute missing data

In [ ]:
from sklearn.impute import KNNImputer
import joblib
imputer = KNNImputer(n_neighbors=5)
imputer.fit(final_data)
joblib.dump(imputer, '../result/00pre-processing/knn_imputer_model.pkl')
df_imputed = pd.DataFrame(imputer.fit_transform(final_data), columns=final_data.columns, index=final_data.index)
df_imputed['e抗原COI'] = df_imputed['e抗原COI'].apply(lambda x: 1 if x > 0.5 else 0)
df_imputed.to_csv("../result/00pre-processing/02no_miss_data.csv",index=False, encoding='utf-8-sig')

# 03 Select Feature

In [ ]:
import pandas as pd
from sklearn.feature_selection import chi2, mutual_info_classif
from scipy.stats import rankdata
data = pd.read_csv("../result/00pre-processing/02no_miss_data.csv")
df = data.copy(deep=True)

df = df[['Gender', 'Age', 'Subtype', 'ALT', 'AST', 'TProt', 'Albumin', 'Globulin', 'GGT', 'DBIL',
       'IBIL', 'AFP', 'DNA load', 'HBsAg', 'HBeAg_COI', 'SFU', 'HBsAg1_T', 'HBsAg2_T', 
       'HBpol1_T', 'HBpol2_T', 'HBx1_T', 'HBx2_T', 'HBeAg1_T', 'HBeAg2_T', 'ThSched', 'ADV', 'ETV', 'PEG_IFN', 'TAF', 'TDF', 
       'TFV', 'TMF', 'UnusedD' ,'6M-Label','12M-Label']]
clinical_features = ['Gender', 'Age', 'ALT', 'AST', 'TProt', 'Albumin', 'Globulin', 'GGT', 'DBIL', 
                     'IBIL', 'AFP', 'DNA load', 'HBsAg', 'HBeAg_COI', ]

def calculate_feature_ranks(x, y, features):
    """
    Calculate the rank of the three ways
    """
    # chi2
    chi2_stat, p_values = chi2(x, y)
    chi2_df = pd.DataFrame({'Feature': features, 'Chi2_Stat': chi2_stat})
    chi2_df['Chi2_Rank'] = len(features) - rankdata(chi2_stat) + 1
    
    # pearson
    correlation = pd.concat([x, y], axis=1).corr()[y.name].drop(y.name)
    corr_df = pd.DataFrame({'Feature': correlation.index, 'Correlation': correlation.values})
    corr_df['Correlation_Rank'] = len(features) - rankdata(abs(correlation.values)) + 1
    
    # mutual info
    mutual_info = mutual_info_classif(x, y, random_state=42)
    mi_df = pd.DataFrame({'Feature': features, 'Mutual_Info': mutual_info})
    mi_df['MI_Rank'] = len(features) - rankdata(mutual_info) + 1
    
    result = chi2_df.merge(corr_df, on='Feature').merge(mi_df, on='Feature')
    result['Avg_Rank'] = (result['Chi2_Rank'] + result['Correlation_Rank'] + result['MI_Rank']) / 3
    result['Final_Rank'] = rankdata(result['Avg_Rank'])
    # print(result)
    
    return result.sort_values('Final_Rank')

x = df[clinical_features]
y_6m = df['6M-Label']
y_12m = df['12M-Label']

# calculate the ranking of the features at two time point 
print("calculate the ranking of the features at 6M...")
rank_6m = calculate_feature_ranks(x, y_6m, clinical_features)

print("calculate the ranking of the features at 12M...")
rank_12m = calculate_feature_ranks(x, y_12m, clinical_features)

merged_ranks = rank_6m[['Feature', 'Final_Rank']].merge(
    rank_12m[['Feature', 'Final_Rank']], 
    on='Feature', 
    suffixes=('_6M', '_12M')
)

# calculate the overall ranking
merged_ranks['Combined_Rank'] = (merged_ranks['Final_Rank_6M'] + merged_ranks['Final_Rank_12M']) / 2
merged_ranks['Final_Combined_Rank'] = rankdata(merged_ranks['Combined_Rank'], method='min')

all_features = merged_ranks

detailed_results = all_features.merge(
    rank_6m[['Feature', 'Chi2_Stat', 'Correlation', 'Mutual_Info', 'Avg_Rank']].rename(
        columns={'Avg_Rank': 'Avg_Rank_6M'}
    ), on='Feature'
).merge(
    rank_12m[['Feature', 'Chi2_Stat', 'Correlation', 'Mutual_Info', 'Avg_Rank']].rename(
        columns={'Avg_Rank': 'Avg_Rank_12M'}
    ), on='Feature', suffixes=('_6M', '_12M')
)

detailed_results.to_excel("../result/00pre-processing/03all_features_combined.xlsx", index=False, engine='openpyxl')

In [ ]:
select_features = ['Gender', 'ALT', 'AST', 'Albumin', 'GGT', 'DBIL', 'IBIL', 'AFP', 'DNA load', 'HBsAg', 'SFU', 'HBsAg1_T', 'HBsAg2_T', 
       'HBpol1_T', 'HBpol2_T', 'HBx1_T', 'HBx2_T', 'HBeAg1_T', 'HBeAg2_T', 'ThSched', 'ADV', 'ETV', 'PEG_IFN', 'TAF', 'TDF', 
       'TFV', 'TMF','6M-Label','12M-Label']
df=df[select_features]
STCF_mapping = {
    "HBsAg1_T": "HBsAg-T(pH>7)", "HBsAg2_T": "HBsAg-T(pH≤7)", 
    "HBpol1_T": "HBpol-T(pH>7)", "HBpol2_T": "HBpol-T(pH≤7)", 
    "HBx1_T": "HBx-T(pH>7)", "HBx2_T": "HBx-T(pH≤7)", 
    "HBeAg1_T": "HBeAg-T(pH>7)", "HBeAg2_T": "HBeAg-T(pH≤7)", 
    'HBeAg COI': "HBeAg",
    'PEG_IFN': "PEG-IFN",
    'SFU': "HBV-T"
    }

df.rename(columns=STCF_mapping, inplace=True)
df.to_csv("../result/00pre-processing/03feature_select_data.csv",index=False,encoding='utf-8-sig')


# 04 Data Normalization

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

df = pd.read_csv("../result/00pre-processing/03feature_select_data.csv")

clinical_features = ['Gender', 'ALT', 'AST', 'Albumin', 'GGT', 'DBIL', 'IBIL', 'AFP', 'DNA load', 'HBsAg']
specific_features = ['HBV-T', 'HBsAg-T(pH>7)', 'HBsAg-T(pH≤7)', 'HBpol-T(pH>7)', 'HBpol-T(pH≤7)', 'HBx-T(pH>7)', 'HBx-T(pH≤7)', 'HBeAg-T(pH>7)', 'HBeAg-T(pH≤7)']
treat_features = ['ThSched', 'ADV', 'ETV', 'PEG-IFN', 'TAF', 'TDF', 'TFV', 'TMF']

df = df[clinical_features+specific_features+treat_features+['6M-Label','12M-Label']]

df_scaler = df.copy(deep=True)
df_scaler.drop(['6M-Label','12M-Label'], axis=1, inplace=True)

minmax_transfer = MinMaxScaler()
df_scaler_minmax = minmax_transfer.fit_transform(df_scaler)
joblib.dump(minmax_transfer, "../result/00pre-processing/scaler.joblib")

df_scaler = pd.DataFrame(df_scaler_minmax, columns=df_scaler.columns)
df_scaler[['6M-Label','12M-Label']] = df[['6M-Label','12M-Label']]

df_scaler.to_csv("../result/00pre-processing/05final_data-minmax.csv", index=False, encoding="utf-8-sig")